In [1]:
import numpy as np
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# import eht 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [21]:
train = pd.read_csv('/home/zhangyd/Data/Kaggle/DigitRecognizer/train.csv')

In [56]:
train.head()

In [5]:
print(train.shape)

(42000, 785)


In [22]:
# save the labels to a Pandas series target
target = train['label']
# Drop the label feature
train = train.drop('label', axis=1)

In [23]:
# standardize the data
from sklearn.preprocessing import StandardScaler
X = train.values
X_std = StandardScaler().fit_transform(X)

/home/zhangyd/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.



In [5]:
# Calculating Eigenvectors and eigenvalues of Cov matrix
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [6]:
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]

In [7]:
# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [8]:
# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # individual explained variance
cum_var_exp = np.cumsum(var_exp) # cumulative explained variance

In [55]:
trace1 = go.Scatter(
    x = list(range(784)), 
    y = cum_var_exp, 
    mode = 'lines+markers', 
    name = 'Cumulative Explained Variance', 
    hoverinfo = cum_var_exp, 
    line = dict(
        shape = 'spline', 
        color = 'goldenrod'
    )
)
trace2 = go.Scatter(
    x = list(range(784)), 
    y = var_exp, 
    mode = 'lines+markers', 
    name = 'individual Explained Variance', 
    hoverinfo = var_exp, 
    line = dict(
        shape = 'liner', 
        color = 'black'
    )
)
fig = tls.make_subplots(insets=[{'cell': (1, 1), 'l': 0.7, 'b': 0.5}], print_grid=True)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.layout.title = 'Explained Variace plots - Full and Zoomed-in'
fig.layout.xaxis = dict(range=[0, 80], title = 'Feature columns')
fig.layout.yaxis = dict(range=[0, 60], title = 'Explained Variance')
fig['data'] += [go.Scatter(x=list(range(784)), y=cum_var_exp, xaxis='x2', yaxis='y2', name='Cumulative Explained Variance')]
fig['data'] += [go.Scatter(x=list(range(784)), y=var_exp, xaxis='x2', yaxis='y2', name='Individual Explained Variance')]
py.iplot(fig, filename='inset example')

In [9]:
# Invoke SKlearn's PCA method
n_components = 30
pca = PCA(n_components=n_components).fit(train.values)
eigenvalues = pca.components_ # 这里估计应该是特征向量，个人理解

In [47]:
n_row = 4
n_col = 7
# plot the first 8 eignenvalues
plt.figure(figsize=(13, 12))
for i in list(range(n_row * n_col)):
    offset = 0
    plt.subplot(n_row, n_col, i + 1)
    plt.imshow(eigenvalues[i].reshape(28, 28), cmap='jet')
    title_text = 'Eigenvalue' + str(i+1)
    plt.title(title_text, size=6.5)
    plt.xticks(())
    plt.yticks(())
plt.show()

In [12]:
# plot some of the numbers
plt.figure(figsize=(14, 12))
for digit_num in range(0, 70):
    plt.subplot(7, 10, digit_num+1)
    grid_data = train.iloc[digit_num].as_matrix().reshape(28, 28) # reshape from 1d to 2d pixel array
    plt.imshow(grid_data, interpolation = 'none', cmap = 'afmhot')
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()

In [24]:
# Delete our earlier created X object
del X
# Taking only the first N rows to speed things up
X = train[:6000].values
del train
# Standardising the values
X_std = StandardScaler().fit_transform(X)

# Call the PCA method with 5 components
pca = PCA(n_components=5)
pca.fit(X_std)
X_5d = pca.transform(X_std) # 将数据降维

# For cluster coloring in our Plotly plots, remember to also restrict the target values
Target = target[:6000]

/home/zhangyd/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.



In [37]:
trace0 = go.Scatter(
    x = X_5d[:, 0], 
    y = X_5d[:, 1], 
    name = Target, 
    hoveron = Target, 
    mode = 'markers', 
    text = Target, 
    showlegend = False, 
    marker = dict(
        size = 8, 
        color = Target, 
        colorscale = 'Jet', 
        showscale = False, 
        line = dict(
            width = 2, 
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [trace0]

layout = go.Layout(
    title = 'Principal Component Analysis (PCA)', 
    hovermode = 'closest', 
    xaxis = dict(
        title = 'First Principal Component', 
        ticklen = 5, 
        zeroline = False, 
        gridwidth = 2, 
    ),
    yaxis = dict(
        title = 'Second Principal Component', 
        ticklen = 5, 
        gridwidth = 2
    ), 
    showlegend = True
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [39]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=9)
X_clustered = kmeans.fit_predict(X_5d)

trace_Kmeans = go.Scatter(x=X_5d[:, 0], y=X_5d[:, 1], mode='markers', 
                         showlegend = False, 
                         marker = dict(
                             size = 8, 
                             color = X_clustered, 
                             colorscale = 'Portland', 
                             showscale = False, 
                             line = dict(
                                 width = 2, 
                                 color = 'rgb(255, 255, 255)'
                             )
                         ))
layout = go.Layout(
    title = 'KMeans Clustering', 
    hovermode = 'closest', 
    xaxis = dict(
        title = 'First Principal Component', 
        ticklen = 5, 
        zeroline = False, 
        gridwidth = 2
    ), 
    yaxis = dict(
        title = 'Second Principal Component', 
        ticklen = 5, 
        gridwidth = 2
    ), 
    showlegend = True
)

data = [trace_Kmeans]
fig1 = dict(data=data, layout=layout)
py.iplot(fig1, filename='svm')

# Linear Discriminant Analysis (LDA) 

In [40]:
from IPython.display import display, Math, Latex

In [42]:
lda = LDA(n_components=5)
# Taking in as second argument the Target as labels
X_LDA_2D = lda.fit_transform(X_std, Target.values)

/home/zhangyd/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning:

Variables are collinear.



In [46]:
traceLDA = go.Scatter(
    x = X_LDA_2D[:, 0], 
    y = X_LDA_2D[:, 1], 
    name = Target, 
    mode = 'markers', 
    text = Target, 
    showlegend = True, 
    marker = dict(
        size = 8, 
        color = Target, 
        colorscale = 'Jet', 
        showscale = False, 
        line = dict(
            width = 2, 
            color = 'rgb(255, 255, 255)'
        ), 
        opacity = 0.8
    )
)
data = [traceLDA]

layout = go.Layout(
    title = 'Linear Discriminant Analysis (LDA)', 
    hovermode = 'closest', 
    xaxis = dict(
        title = 'First Linear Discriminant', 
        ticklen = 5, 
        zeroline = False, 
        gridwidth = 2
    ),
    yaxis = dict(
        title = 'Second Linear Discriminant', 
        ticklen = 5, 
        gridwidth = 2
    ), 
    showlegend = False
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

# T-SNE (t-Distributed Stochastic Neighbour Embedding) 

In [52]:
# Invoking the t-SNE method
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(X_std)

In [54]:
traceTSNE = go.Scatter(
    x = tsne_results[:, 0], 
    y = tsne_results[:, 1], 
    name = Target, 
    hoveron = Target, 
    mode = 'markers', 
    text = Target, 
    showlegend = True, 
    marker = dict(
        size = 8, 
        color = Target, 
        colorscale = 'Jet', 
        showscale = False, 
        line = dict(
            width = 2, 
            color = 'rgb(255, 255, 255)'
        ), 
        opacity = 0.8
    )
)
data = [traceTSNE]

layout = dict(
    title = 'TSNE (t-Distributed Stochastic Neighbour Embedding)', 
    hovermode = 'closest', 
    yaxis = dict(zeroline=False), 
    xaxis = dict(zeroline=False), 
    showlegend = False
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')